In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [2]:
sample_submission = pd.read_csv("sample_submission.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

y_train = train["label"]
x_train = train.drop(labels = ["label"],axis = 1) 

In [3]:
x_train = x_train/255.0
test = test/255.0

x_train = x_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [4]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.2,random_state=5)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)),
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'),
    tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = "relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation= "softmax")
])

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2)  # randomly shift images vertically (fraction of total height)

In [8]:
batch_size = 80
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = 100, validation_data = (x_val,y_val),
                              verbose = 2, steps_per_epoch = 100)

Train for 100 steps, validate on 8400 samples
Epoch 1/100
100/100 - 7s - loss: 1.5313 - accuracy: 0.4664 - val_loss: 0.2923 - val_accuracy: 0.9136
Epoch 2/100
100/100 - 3s - loss: 0.5903 - accuracy: 0.8109 - val_loss: 0.1789 - val_accuracy: 0.9444
Epoch 3/100
100/100 - 3s - loss: 0.4005 - accuracy: 0.8788 - val_loss: 0.1269 - val_accuracy: 0.9620
Epoch 4/100
100/100 - 3s - loss: 0.3125 - accuracy: 0.9079 - val_loss: 0.0880 - val_accuracy: 0.9731
Epoch 5/100
100/100 - 3s - loss: 0.2669 - accuracy: 0.9215 - val_loss: 0.0952 - val_accuracy: 0.9705
Epoch 6/100
100/100 - 3s - loss: 0.2215 - accuracy: 0.9341 - val_loss: 0.0660 - val_accuracy: 0.9802
Epoch 7/100
100/100 - 3s - loss: 0.1953 - accuracy: 0.9413 - val_loss: 0.0572 - val_accuracy: 0.9824
Epoch 8/100
100/100 - 3s - loss: 0.1946 - accuracy: 0.9441 - val_loss: 0.0783 - val_accuracy: 0.9749
Epoch 9/100
100/100 - 3s - loss: 0.1711 - accuracy: 0.9526 - val_loss: 0.0539 - val_accuracy: 0.9836
Epoch 10/100
100/100 - 3s - loss: 0.1693 - ac

In [9]:
results = model.predict(test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("cnn_mnist.csv",index=False)
